# Lindhardt susceptibility $\chi_0$ for a non interacting electron gas


HERE WE RECALL WAHT LINDHARD FUNCTION IS



The two-particle response of the non-interacting system is given by the Lindhardt susceptibility $\chi_0$. Diagrammatically it is a bubble diagram of two single-particle Green's function, that is most efficiently evaluated in imaginary time.

$$ \chi_0(\tau, r) \equiv 2 g_0(\tau, r) g_0(\beta - \tau, -r) $$

after evaluating the imaginary time bubble we transform $\chi_0$ back to momentum and frequency space, i.e.


$$ \chi_0(i\omega_n, r) \equiv \mathcal{F}_{\tau \rightarrow i\omega_n} \big\{ \chi_0(\tau, r) \big\}$$

$$ \chi_0(i\omega_n, k) \equiv \mathcal{F}_{r \rightarrow k} \big\{ \chi_0(i\omega_n, r) \big\}$$


While it is possible to do this computation in pure Python, it is relatively slow, due to the double loop in $r$ and $\tau$. 

Here we illustrate the C++ layer of the TRIQS library, using the TRIQS/cpp2py tool to wrap Python and C++
in a simple case.

The function bubble below computes the Lindhardt function from Eq (XX).
The following cell compiles the function "bubble" in C++, exposes ("wraps") it to Python and loads it
in the current scope.


In [ ]:
%reload_ext cpp2py.magic

May take a few seconds to compile ...

In [ ]:
%%cpp2py -C pytriqs
#include <triqs/gfs.hpp>
using namespace triqs::gfs;

// The type of a Green function : (omega,k) -> Complex number
using g_w_k_type = gf_view<cartesian_product<imfreq, brillouin_zone>, scalar_valued>;

g_w_k_type bubble(g_w_k_type g) {
    
    // Fourier Transformation of k, \omega to obtain g(t,r)
    auto gtr = make_gf_from_fourier<0,1>(g);
    
    // The mesh of gtr is a cartesian product mt x mr. We decompose it.
    auto [mt, mr] = gtr.mesh();
    
    // The inverse temperature from the mesh
    double beta = mt.domain().beta;
    
    // A new mesh for chi, with a bosonic statistics, but same size as mt.
    auto mtb = gf_mesh<imtime>{beta, Boson, mt.size()};
    
    // Build chi (tau, r) with this new mesh.
    auto chi0 = gf<cartesian_product<imtime, cyclic_lattice>, scalar_valued>{{mtb, mr}};

    // we fill chi : chi(tau, r) = g(beta - tau, -r) * g(tau, r)
    for (auto const &r : mr)      
        for (auto const &t : mtb) 
            chi0[t, r] = gtr(beta - t, -r) * gtr(t, r); 

    // Fourier transform back to k, \omega space and return
    return make_gf_from_fourier<0,1>(chi0);
}

# Static Lindhardt susceptibility $\chi_0(\omega=0, k)$

The square lattice with only nearest-neighbour hopping $t$ has a property of "perfect nesting", meaning that large parts of the fermi surface are wrapped on-to each other by a single momentum transfer $k$. Go back to the fermi-surface polot of $\epsilon_k$ and determine this peculiar momentum vector.

The "perfect nesting" greatly enhances the particle-hole susceptibility of the system and the Static Lindhardt susceptibility $\chi_0(\omega=0, k)$ has a dominant peak at this momentum.

TODO : 

- Grab the Gf from HDF5 (to pass from a notebook to the next and keep the notebook simple !)

- Call Lindhard to obtain chi(q, omega)

- Plot chi(k, 0) with a slice
  and other plots.
  
- We save the function in HDF5 for later reuse.  
  
- Physics discussion ?

In [ ]:
from pytriqs.lattice import BravaisLattice, BrillouinZone
from pytriqs.gf import MeshBrillouinZone, MeshImFreq, Gf, MeshProduct
from math import cos
from pytriqs.gf import *
import numpy as np

beta = 1./0.75
t = 1.0 

n_k = 128
n_w = 128

# Two unit vectors in R3
BL = BravaisLattice([(1, 0, 0), (0, 1, 0)])

BZ = BrillouinZone(BL)

# A regular mesh on the BZ with 30 x 30 points
kmesh = MeshBrillouinZone(BZ, n_k = n_k)

# Frequency mesh
wmesh = MeshImFreq(beta=beta, S='Fermion', n_max=n_w)

g = Gf(mesh = MeshProduct(wmesh, kmesh), target_shape = [])

iGamma = 0.01 * 1j

def eps(k):
    return -2 * t* (np.cos(k[0]) + np.cos(k[1]))

# NB : loop is a bit slow in python ...
for k in g.mesh[1]:
    for w in g.mesh[0]:
        g[w,k] = 1/(w - eps(k) + iGamma)


In [ ]:
chi0_wk = bubble(g)
print chi0_wk

In [ ]:
from pytriqs.plot.mpl_interface import oplot, oplotr, oploti, plt

chi0_w0k = chi0_wk[Idx(0), :]

k = np.linspace(0, 2*np.pi, kmesh.linear_dims[0]+1, endpoint=True)
kx, ky = np.meshgrid(k, k)

chi = lambda kx, ky: chi0_w0k([kx,ky,0]).real

plt.figure(figsize=(3.25*2, 5))
plt.title('Static Lindhardt susceptibility $\chi_0(\omega=0, k)$')
plt.imshow(np.vectorize(chi)(kx,ky))

#plt.imshow(data[:, :, 0].real, cmap=plt.get_cmap('terrain_r'), origin='lower', 
#           vmin=0, extent=extent_k)
plt.colorbar(); plt.tight_layout()
#plt.savefig('figure_chi0_w0k.pdf')
plt.xlabel(r'$k_x/(2\pi)$')
plt.ylabel(r'$k_y/(2\pi)$')


# Here is a 3d plot. With "matplotlib inline" one cannot interact
from mpl_toolkits.mplot3d import axes3d, Axes3D #<-- Note the capitalization! 
fig = plt.figure()

ax = Axes3D(fig) #<-- Note the difference from your original code...
ax.plot_surface(kx, ky, np.vectorize(chi)(kx,ky), cmap="jet")
ax.set_xlabel(r"$k_x$")
ax.set_ylabel(r"$k_y$")
ax.set_zlabel(r"$\chi^{0}$")



In [ ]:
from pytriqs.archive import HDFArchive
with HDFArchive("chi0.h5") as R:
    R['chi0_wk'] = chi0_wk

In [ ]:
from k_space_viz import get_rel_k_interpolator

interp = get_rel_k_interpolator(
    chi0_wk[Idx(0), :][0, 0].data, kmesh, H_0.bz, n_k,
    extend_boundary=True, interpolator='linear2D')
chi0_plot = interp(k_path_vecs[:, :2])


plt.figure(figsize=(3.25*3, 8))
plt.title('Static Lindhardt susceptibility $\chi_0(\omega=0, k)$')
plt.plot(k_plot, chi0_plot.real)

plt.grid(); plt.axes().set_xticks(K_plot)
plt.xlim([K_plot.min(), K_plot.max()])
plt.axes().set_xticklabels([r'$\Gamma$',r'$X$',r'$M$',r'$\Gamma$'])
plt.ylabel(r'$\epsilon_k$'); plt.tight_layout()
plt.savefig('figure_chi0_k_bandpath.pdf')